In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

# Loading Data

In [ ]:
items = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/shops.csv')
sales_train = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
sample_submission = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')
item_cat = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv')

# Reading the Data

In [ ]:
print('ITEMS')
display(items.head())

print('ITEM CATEGORIES')
display(item_cat.head())

print('SHOPS')
display(shops.head())

print('SALES TRAIN')
display(sales_train.head())

print('TEST')
display(test.head())

print('SAMPLE SUBMISSION')
display(sample_submission.head())

In [ ]:
# Exploring Sample train dataset
sales_train.info()

In [ ]:
# Checking missing values for sales train and test dataset
print('Sales train')
display(sales_train.isnull().sum())

print('test')
display(test.isnull().sum())

*No Missing values found*

In [ ]:
print('Sales train')
display(sales_train.describe())

print('Test')
display(test.describe())

*Sales train dataset contains negative values in feature item_price and  item_cnt_day.*

# Exploring the Dataset

In [ ]:
# Removing duplicates from Sales train dataset
subset = ['date','date_block_num','shop_id','item_id','item_cnt_day']
sales_train.duplicated(subset=subset).value_counts()

In [ ]:
sales_train.drop_duplicates(subset=subset,inplace=True)

In [ ]:
#Checking for negative values in item_price and item_cnt_day and dropping them as it contains only 1 negative value.
print('Negative values in item_price')
display(sales_train[sales_train['item_price']<0])

print('Negative values in item_cnt_day')
display(sales_train[sales_train['item_cnt_day']<0])


In [ ]:
sales_train = sales_train[sales_train['item_price']>0]
sales_train = sales_train[sales_train['item_cnt_day']>0]


# Data Visualization

In [ ]:
# Finding Outliers for item price and item_cnt_day
sns.boxplot(sales_train['item_price'])

In [ ]:
sns.boxplot(sales_train['item_cnt_day'])

In [ ]:
#Dropping Ouliers
def drop_out(df,feature,high_percentile = .99):
    df_shape = df.shape[0]         #sales train df size before dropping
    max_val = df[feature].quantile(high_percentile)      #Percentile value
    print('Dropping Outliers for ... {}'.format(feature))
    df = df[df[feature] < max_val]
    print(str(df_shape - df.shape[0]) + ' ' + feature + ' values over ' + str(max_val) + ' have been removed' )
    return df
    

In [ ]:
#Dropping outliers for item_price
sales_train = drop_out(sales_train,'item_price')

In [ ]:
#Dropping outliers for item_cnt_day
sales_train = drop_out(sales_train,'item_cnt_day')

In [ ]:
# Creating new dataframe with item_price feature group by shop_id and item_id to get price for each item per shop.
# We can use this dataframe to create item_price feature for the test dataset.

In [ ]:
shop_price_df = sales_train[['shop_id','item_id','item_price']]
shop_price_df = shop_price_df.groupby(['shop_id','item_id']).apply(lambda df: df['item_price'][-2:].mean())
shop_price_df = shop_price_df.to_frame(name='item_price')
shop_price_df

In [ ]:
# Merge this dataframe with test dataframe to create item_price feature in test dataset.
test = pd.merge(test,shop_price_df,how='left', left_on=['shop_id','item_id'], right_on=['shop_id','item_id'])
test.head()

In [ ]:
#check null values
test['item_price'].isnull().sum()      

* Item price in test dataset contains null values.fill this by creating more features from item_categories

In [ ]:
#Split date feature to month and year from sales_train dataset
sales_train['month'] = [date.split('.')[1] for date in sales_train['date']]
sales_train['year'] = [date.split('.')[2] for date in sales_train['date']]

sales_train.drop(['date','date_block_num'],axis=1,inplace=True)

#create month and year features fot test dataset
test['month'] = 11
test['year'] = 2015

In [ ]:
#change item_cnt_day to item_cnt_month 
sales_train_monthly = sales_train.groupby(['year','month','shop_id','item_id'],
                                          as_index=False)[['item_cnt_day']].sum()
sales_train_monthly.rename(columns={'item_cnt_day':'item_cnt_month'},inplace=True)

sales_train_monthly = pd.merge(sales_train_monthly,shop_price_df,how='left',
                               left_on=['shop_id','item_id'],right_on=['shop_id','item_id'])
sales_train_monthly.head()

In [ ]:
sales_train = sales_train_monthly
sales_train.head()

In [ ]:
test = test.reindex(columns=['ID','year','month','shop_id','item_id','item_price'])
test.head()

# Exploring other Dataset
* 1. Item_categories

In [ ]:
#Extracting main categories 
item_cat['main categories'] = [x.split('-')[0] for x in item_cat['item_category_name']]

# Some items dont have sub category for them we will use None as a sub category
sub_cat=[]
for i in range(len(item_cat)):
    try:
        sub_cat.append(item_cat['item_category_name'][i].split('-')[1])
    except IndexError as e:
        sub_cat.append('None')

item_cat['sub categories'] = sub_cat
item_cat.drop(['item_category_name'],axis=1,inplace=True)
item_cat.head()

* 2. Items dataset

In [ ]:
items = pd.merge(items,item_cat,how='left')

#drop item_name and item_category_id
items.drop(['item_name','item_category_id'],axis=1,inplace=True)
items.head()

In [ ]:
# Merge items to test and sales_train dataset
sales_train = pd.merge(sales_train,items,how='left')
test = pd.merge(test,items,how='left')

* 3. Shops dataset

In [ ]:
from string import punctuation

#replace all the punctuations from shop_name feature
shops['shop_name_cleaned'] = shops['shop_name'].apply(lambda s:''.join([x for x in s if x not in punctuation]))

#Extract shop city name
shops['shop_city'] = shops['shop_name_cleaned'].apply(lambda s:s.split()[0])

#Extract shop type
shops['shop_type'] = shops['shop_name_cleaned'].apply(lambda s:s.split()[1])

#Extract shop name
shops['shop_name'] = shops['shop_name_cleaned'].apply(lambda s:s.split()[2:])

#Dropping shop_name_cleaned
shops.drop(['shop_name_cleaned'],axis=1,inplace=True)

shops.head()

In [ ]:
#Merging sales_train and test dataset with shops dataset
sales_train = pd.merge(sales_train,shops,how='left')
test = pd.merge(test,shops,how='left')

In [ ]:
print('SALES TRAIN')
display(sales_train.head())

print('TEST')
display(test.head())

* Fill Missing values in item_price

In [ ]:
#fill missing values with median of each main_category and sub_category
test['item_price'] = test.groupby(['main categories',
                                   'sub categories'])['item_price'].apply(lambda df: df.fillna(df.median()))
test['item_price'].isnull().sum()

In [ ]:
# Fill missing values with median of each sub category
test['item_price'] = test.groupby(['sub categories'])['item_price'].apply(lambda df : df.fillna(df.median()))
test['item_price'].isnull().sum()

In [ ]:
test[test['item_price'].isnull()]

* All the null item_price data contains same main categories (PC) and sub catogories (Гарнитуры/Наушники) value. This main and sub categories are not present in test dataset but are present in sales_train dataset.

In [ ]:
#fill missing values with median of main_category and sub_category from sales_train dataset

filler = sales_train[(sales_train['main categories'] == 'PC') & (sales_train['sub categories'] == 'Гарнитуры/Наушники')]['item_price'].median()
filler = 0
test['item_price'].fillna(filler, inplace=True)

In [ ]:
test['item_price'].isnull().sum()

In [ ]:
# Clipping the item_cnt_month value to [0,20] range
sales_train['item_cnt_month'] = sales_train['item_cnt_month'].clip(0,20)

In [ ]:
# Defining a target array and dropping it from sales_train dataset
target_aaray = sales_train['item_cnt_month']
sales_train.drop(['item_cnt_month'],axis=1,inplace=True)

#Dropping ID column from test dataset
test_id = test['ID']
test.drop(['ID'],axis=1,inplace=True)


In [ ]:
#Drop shop_id and item_id from sales_train and test dataset
sales_train.drop(['shop_id','item_id'],axis=1,inplace=True)
test.drop(['shop_id','item_id'],axis=1,inplace=True)

In [ ]:
#Reducing memory usage
def downcast_dtypes(df):
    # Selecting columns to downcast
    flt_cols = [c for c in df if df[c].dtype == 'float64']
    int_cols = [d for d in df if df[d].dtype == 'int64']
    
    #Downcasting
    df[flt_cols] = df[flt_cols].astype(np.float16)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [ ]:
downcast_dtypes(sales_train)
downcast_dtypes(test)

print('SALES TRAIN')
display(sales_train.head())

print('TEST')
display(test.head())

In [ ]:
sales_train.info()

In [ ]:
#Check for null values in test and sales_train dataset
print('Missing data in sales train: ',sales_train.isnull().any().sum())
print('Missing data in test: ',test.isnull().any().sum())

In [ ]:
#Normality test function
def norm_test(data,alpha=0.05):
    from scipy import stats
    statistic,p_val = stats.normaltest(data)
    
    #null hypothesis: array comes from a normal distribution
    if p_val < alpha:  
        #The null hypothesis can be rejected
        is_normal_dist = False
    else:
        #The null hypothesis cannot be rejected
        is_normal_dist = True
    
    return is_normal_dist

In [ ]:
# Check normality of all numerical features and change them to normal distribution
for i in sales_train.columns:
    if sales_train[i].dtype != 'object':
        if norm_test(sales_train[i]) == False:
            sales_train[i] = np.log1p(sales_train[i])
            test[i] = np.log1p(test[i])

In [ ]:
target_aaray = np.log1p(target_aaray)

In [ ]:
#ENCODING
ord_enc = OrdinalEncoder()
arr = sales_train.to_numpy()
#x = sales_train.to_numpy(dtype=str)
print(arr)
X = ord_enc.fit_transform(arr)
Y = target_aaray

#X_predict = ord_enc.fit_transform(test.to_numpy())

* Train test split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(sales_train,target_aaray,test_size=0.1,random_state=0)

# Model creation 
* We will use XGBRegressor model to predict total sales for every product and store

In [ ]:
#Create a model
xg_model = XGBRegressor()

#Fitting model
xg_model.fit(x_train, y_train, eval_metric="rmse", eval_set=[(x_train, y_train), (x_test, y_test)], verbose=True, early_stopping_rounds = 20)